In [1]:
import os
import pandas as pd

pd.set_option('display.max_columns', None)

In [19]:
data_folder = 'datos'
subfolders = [f.path for f in os.scandir(data_folder) if f.is_dir()]

for subfolder in subfolders:
    # Get the name of the subfolder
    folder_name = 'matriculas'
    
    # Get the list of CSV files in the subfolder
    csv_files = [f.path for f in os.scandir(subfolder) if f.is_file() and f.path.endswith('.csv')]
    
    # Read and concatenate all CSV files in the subfolder
    df_list = [pd.read_csv(csv_file, low_memory=False) for csv_file in csv_files]
    combined_df = pd.concat(df_list, ignore_index=True)
    
    # Save the combined DataFrame to a new CSV file with the name of the folder in the data folder
    combined_df.to_csv(f'{data_folder}/{folder_name}.csv', index=False)

    # save the combined datafram to a variable with the name of the folder
    globals()[folder_name] = combined_df.copy()

del combined_df
del csv_files	 
del df_list	 
del folder_name
del subfolder

## Ejecutar a partir de aquí para más rápido

In [20]:
# i want to execute this only if the variable matriculas is not defined
if 'matriculas' not in locals():
    matriculas = pd.read_csv('datos/matriculas.csv', low_memory=False)

In [21]:
if 'IMPORTANTE' in matriculas.columns:
    matriculas.drop(columns=['IMPORTANTE'], inplace=True)

In [22]:
print(matriculas.shape, " - ", matriculas.columns)
print(matriculas.head())

(1314854, 79)  -  Index(['cod_universidad', 'des_universidad', 'curso_academico',
       'curso_academico_acceso_SUE', 'curso_academico_inicio',
       'cod_pais_nacionalidad', 'des_pais_nacionalidad',
       'lat_pais_nacionalidad', 'lon_pais_nacionalidad',
       'cod_continente_nacionalidad', 'des_continente_nacionalidad',
       'cod_agregacion_paises_nacionalidad',
       'des_agregacion_paises_nacionalidad', 'cod_pais_residencia',
       'des_pais_residencia', 'lat_pais_residencia', 'lon_pais_residencia',
       'cod_continente_residencia', 'des_continente_residencia',
       'cod_agregacion_paises_residencia', 'des_agregacion_paises_residencia',
       'cod_comunidad_residencia', 'des_comunidad_residencia',
       'cod_provincia_residencia', 'des_provincia_residencia',
       'cod_municipio_residencia', 'des_municipio_residencia',
       'lat_municipio_residencia', 'lon_municipio_residencia',
       'cod_comunidad_curso', 'des_comunidad_curso', 'cod_provincia_curso',
       'des

# Limpieza de NaNs

In [23]:
# i want to see all columns with nans and the number of nans, scrollable
print(matriculas.isnull().sum())

cod_universidad                     0
des_universidad                     0
curso_academico                     0
curso_academico_acceso_SUE     271314
curso_academico_inicio              0
                               ...   
ind_tesis_leida_curso         1229172
ind_mencion_internacional     1305083
ind_mencion_industrial        1305544
cod_calificacion_tesis        1305085
des_calificacion_tesis        1305086
Length: 79, dtype: int64


In [24]:
# eliminamos doctorados
matriculas = matriculas[matriculas['cod_tipo_estudio'] != 'D']

In [25]:
# fill nans in curso_academico_acceso_SUE with the value of curso_academico_matricula

In [26]:
matriculas.loc[:, 'curso_academico'] = matriculas['curso_academico'].astype(str)
matriculas.loc[:, 'curso_academico_inicio'] = matriculas['curso_academico_inicio'].astype(str)
matriculas.loc[:, 'curso_academico_acceso_SUE'] = matriculas['curso_academico_acceso_SUE'].astype(str)

benjamines = matriculas[
    (matriculas['curso_academico'] == matriculas['curso_academico_inicio']) & 
    (matriculas['curso_academico'] == matriculas['curso_academico_acceso_SUE'])
]

alevines = matriculas[
    (matriculas['curso_academico'] == matriculas['curso_academico_inicio']) & 
    (matriculas['curso_academico'] != matriculas['curso_academico_acceso_SUE'])
]

print("Alumnos que han entrado por primera vez al SUE, primera matricula (benjamines) shape:", benjamines.shape)
print("Alumnos en primer año que ya han estado previamente en el SUE (alevines) shape:", alevines.shape)

Alumnos que han entrado por primera vez al SUE, primera matricula (benjamines) shape: (241427, 79)
Alumnos en primer año que ya han estado previamente en el SUE (alevines) shape: (91251, 79)


In [27]:
# subdivide alevines in two groups: cod_tipo_estudio == 'G' and cod_tipo_estudio == 'M', and benjamines as well
alevines_G = alevines[alevines['cod_tipo_estudio'] == 'G']
alevines_M = alevines[alevines['cod_tipo_estudio'] == 'M']
benjamines_G = benjamines[benjamines['cod_tipo_estudio'] == 'G']
benjamines_M = benjamines[benjamines['cod_tipo_estudio'] == 'M']

In [28]:
columnas_grados = [
    "cod_tipo_titulo_acceso_master",
    "des_tipo_titulo_acceso_master",
    "nota_media_final",
    "cod_pais_univ_procedencia",
    "des_pais_univ_procedencia",
    "lat_pais_univ_procedencia",
    "lon_pais_univ_procedencia",
    "cod_continente_univ_procedencia",
    "des_continente_univ_procedencia",
    "cod_agregacion_paises_univ_procedencia",
    "des_agregacion_paises_univ_procedencia",
    "ind_tesis_leida_curso",
    "ind_mencion_internacional",
    "ind_mencion_industrial",
    "cod_calificacion_tesis",
    "des_calificacion_tesis"]

benjamines_G = benjamines_G.drop(columns=columnas_grados).copy()
alevines_G = alevines_G.drop(columns=columnas_grados).copy()

In [29]:
columnas_master = [
    "cod_pais_univ_procedencia",
    "des_pais_univ_procedencia",
    "lat_pais_univ_procedencia",
    "lon_pais_univ_procedencia",
    "cod_continente_univ_procedencia",
    "des_continente_univ_procedencia",
    "cod_agregacion_paises_univ_procedencia",
    "des_agregacion_paises_univ_procedencia",
    "ind_tesis_leida_curso",
    "ind_mencion_internacional",
    "ind_mencion_industrial",
    "cod_calificacion_tesis",
    "des_calificacion_tesis"]

benjamines_M = benjamines_M.drop(columns=columnas_master).copy()
alevines_M = alevines_M.drop(columns=columnas_master).copy()

In [30]:
# columns that have more than 1 nan in benjamines
print("Benjamines de Grado", benjamines_G.isnull().sum()[benjamines_G.isnull().sum() > 0])
print("Benjamines de Master", benjamines_M.isnull().sum()[benjamines_M.isnull().sum() > 0])

Benjamines de Grado cod_pais_nacionalidad                    1
des_pais_nacionalidad                    1
lat_pais_nacionalidad                    1
lon_pais_nacionalidad                    1
cod_continente_nacionalidad              1
des_continente_nacionalidad              1
cod_agregacion_paises_nacionalidad       1
des_agregacion_paises_nacionalidad       1
cod_comunidad_residencia              1782
des_comunidad_residencia              1835
cod_provincia_residencia              1352
des_provincia_residencia              1835
cod_municipio_residencia                15
des_municipio_residencia               966
lat_municipio_residencia              2824
lon_municipio_residencia              2824
cod_comunidad_curso                   1034
des_comunidad_curso                   1059
cod_provincia_curso                    728
des_provincia_curso                   1059
cod_municipio_curso                     17
des_municipio_curso                   1045
lat_municipio_curso               

In [31]:
print("Alevines de Grado", alevines_G.isnull().sum()[alevines_G.isnull().sum() > 0])
print("Alevines de Master", alevines_M.isnull().sum()[alevines_M.isnull().sum() > 0])

Alevines de Grado des_pais_nacionalidad                   1
lat_pais_nacionalidad                   1
lon_pais_nacionalidad                   1
cod_continente_nacionalidad             1
des_continente_nacionalidad             1
cod_agregacion_paises_nacionalidad      1
des_agregacion_paises_nacionalidad      1
cod_comunidad_residencia              516
des_comunidad_residencia              532
cod_provincia_residencia              450
des_provincia_residencia              532
cod_municipio_residencia               16
des_municipio_residencia              321
lat_municipio_residencia              847
lon_municipio_residencia              847
cod_comunidad_curso                   184
des_comunidad_curso                   190
cod_provincia_curso                   142
des_provincia_curso                   190
cod_municipio_curso                     5
des_municipio_curso                   156
lat_municipio_curso                   340
lon_municipio_curso                   340
cod_titulacion  

In [32]:
print("Benjamines Grados columns:", benjamines_G.columns)
print("Benjamines Master columns:", benjamines_M.columns)
print("Alevines Grados columns:", alevines_G.columns)
print("Alevines Master columns:", alevines_M.columns)

Benjamines Grados columns: Index(['cod_universidad', 'des_universidad', 'curso_academico',
       'curso_academico_acceso_SUE', 'curso_academico_inicio',
       'cod_pais_nacionalidad', 'des_pais_nacionalidad',
       'lat_pais_nacionalidad', 'lon_pais_nacionalidad',
       'cod_continente_nacionalidad', 'des_continente_nacionalidad',
       'cod_agregacion_paises_nacionalidad',
       'des_agregacion_paises_nacionalidad', 'cod_pais_residencia',
       'des_pais_residencia', 'lat_pais_residencia', 'lon_pais_residencia',
       'cod_continente_residencia', 'des_continente_residencia',
       'cod_agregacion_paises_residencia', 'des_agregacion_paises_residencia',
       'cod_comunidad_residencia', 'des_comunidad_residencia',
       'cod_provincia_residencia', 'des_provincia_residencia',
       'cod_municipio_residencia', 'des_municipio_residencia',
       'lat_municipio_residencia', 'lon_municipio_residencia',
       'cod_comunidad_curso', 'des_comunidad_curso', 'cod_provincia_curso',
  

In [33]:
benjamines_G.to_csv('datos/limpios/benjamines_G.csv', index=False)
alevines_G.to_csv('datos/limpios/alevines_G.csv', index=False)
benjamines_M.to_csv('datos/limpios/benjamines_M.csv', index=False)
alevines_M.to_csv('datos/limpios/alevines_M.csv', index=False)